In [1]:
!nvidia-smi

Wed May 15 00:33:02 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.52       Driver Version: 457.52       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   51C    P8     4W /  N/A |    749MiB /  4096MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [2]:
torch.cuda.device_count()

1

In [3]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [4]:
# 查看张量的存储设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [18]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [19]:
Y = torch.rand(2, 3, device=try_gpu(10))
Y

tensor([[0.1949, 0.8316, 0.0135],
        [0.1468, 0.9901, 0.9366]])

In [20]:
# 将张量Y复制到GPU一份
Y_gpu = Y.cuda(0)
print(X)
print(Y_gpu)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[0.1949, 0.8316, 0.0135],
        [0.1468, 0.9901, 0.9366]], device='cuda:0')


In [14]:
# cpu张量和gpu张量无法直接计算
Y + Y_gpu

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [21]:
X + Y_gpu

tensor([[1.1949, 1.8316, 1.0135],
        [1.1468, 1.9901, 1.9366]], device='cuda:0')

In [23]:
# 变量已经在gpu中，torch不会复制并分配新的内存
Y_gpu.cuda(0) is Y_gpu

True

In [24]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [25]:
net(X)

tensor([[1.2934],
        [1.2934]], device='cuda:0', grad_fn=<AddmmBackward>)

In [26]:
net[0].weight.data.device

device(type='cuda', index=0)

In [27]:
import time

startTime1=time.time()
for i in range(100):
    A = torch.ones(500,500)
    B = torch.ones(500,500)
    C = torch.matmul(A,B)
endTime1=time.time()

startTime2=time.time()
for i in range(100):
    A = torch.ones(500,500,device=try_gpu())
    B = torch.ones(500,500,device=try_gpu())
    C = torch.matmul(A,B)
endTime2=time.time()

print('cpu计算总时长:', round((endTime1 - startTime1)*1000, 2),'ms')
print('gpu计算总时长:', round((endTime2 - startTime2)*1000, 2),'ms')

cpu计算总时长: 258.17 ms
gpu计算总时长: 38.15 ms
